In [1]:
import vampyr.vampyr3d as vp
from math import pi
import matplotlib.pyplot as plt
from numpy import arange
from ipywidgets import interact

In [2]:
min_scale = 0
prec = 1e-7

corner = [0, 0, 0]
boxes = [1, 1, 1]
world = vp.BoundingBox(min_scale, corner, boxes)

order = 3
basis = vp.InterpolatingBasis(order) 
MRA = vp.MultiResolutionAnalysis(world, basis, 25)

In [3]:
beta = 1.0e+2
alpha = (beta/pi)**(3/2)
rho_nuc = vp.GaussFunc(beta, alpha, pos=[0.6, 0.5, 0.5])

In [4]:
def get_func(scale):
    rho_nuc_tree = vp.FunctionTree(MRA)
    # vp.build_grid(rho_nuc_tree, rho_nuc)
    vp.project(prec, rho_nuc_tree, rho_nuc, scale)
    return rho_nuc_tree


In [5]:
funcs = [get_func(order) for order in range(8)]

In [6]:
x = arange(0, 1, 0.001)

In [7]:
y = [[func.evalf([x_i, 0.5, 0.5]) for x_i in x] for func in funcs]
exact = [rho_nuc.evalf([x_i, 0.5, 0.5]) for x_i in x]

In [8]:
@interact(i=(0, 7))
def plot_funcs(i):
    plt.subplot(2, 2, 1)
    plt.axis((0,1,-50,250))
    plt.title('Approximation of Gaussian', color='white')
    plt.plot(x, y[i])
    if i > 0:
        diff = [n-nm1 for n, nm1 in zip(y[i], y[i-1]) ]
        plt.subplot(2, 2, 2)
        #plt.axis((0, 1, -50, 50))
        #plt.yscale('log')
        plt.title(f'F{i} refinements - F{i-1} refinements', color='white')
        plt.plot(x, diff)
    diff_exact = [abs(approx - ex) for approx, ex in zip(y[i], exact)] 
    plt.subplot(2, 2, 3)
    plt.title(f'Error after {i} refinements', color='white')
    plt.plot(x, diff_exact)
    
    plt.subplots_adjust(top=1.0, bottom=0.1, right=2, wspace=0.3)

interactive(children=(IntSlider(value=3, description='i', max=7), Output()), _dom_classes=('widget-interact',)…